In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset

In [12]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Set device to CUDA if available, else CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the pre-trained model and tokenizer
model_path = ".....USD\\Natural Language Processing\\Final Project\\Model"
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-medium')
tokenizer.pad_token = tokenizer.eos_token

# Chat class for conversation
class Chat:
    def __init__(self, chatbot, max_response_len=12):
        self.chatbot = chatbot
        self.messages = []
        self.max_response_len = max_response_len
        self.temperature = 0.4  # Default temperature
        self.top_p = 0.6  # Default top_p

    # (1) Send a message in multi-turn conversation
    def send(self, text: str):
        # Add user message to the conversation
        self.messages.append(f"User: {text}")
        
        # Create the conversation prompt with proper role annotations
        prompt = '<|endoftext|>'.join(self.messages) + '<|endoftext|>'
        
        # Trim history if too long (keeping recent turns)
        max_tokens = 356 # Adjust based on your model's max token limit
        prompt_tokens = self.chatbot.tokenizer.encode(prompt)
        if len(prompt_tokens) > max_tokens:
            # Keep the most recent context, discard older conversation
            prompt_tokens = prompt_tokens[-max_tokens:]
            prompt = self.chatbot.tokenizer.decode(prompt_tokens)

        # Generate a response, passing the current mood (temperature, top_p)
        response = self.chatbot.generate(prompt, max_length=self.max_response_len, temperature=self.temperature, top_p=self.top_p)
        
        # Add bot response to the conversation
        self.messages.append(f"Bot: {response}")
        
        return response

    # (2) Set a specific topic for the conversation
    def set_topic(self, topic: str):
        self.messages.append(f"Topic: {topic}")

    # (4) Reset the conversation history
    def reset_conversation(self):
        self.messages = []

    # (5) Set chatbot mood by adjusting temperature and top_p values
    def set_mood(self, temperature: float, top_p: float):
        self.temperature = temperature
        self.top_p = top_p

class ChatBot:
    def __init__(self, model_path: str, device=None):
        if not device:
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.device = device
        self.model = GPT2LMHeadModel.from_pretrained(model_path).to(self.device)
        self.tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-medium')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    # (3) Generate response based on conversation context and mood
    def generate(self, text: str, max_length: int = None, temperature: float = 0.3, top_p: float = 0.7) -> str:
        if not max_length:
            # Determine max length based on text complexity or conversation context
            max_length = min(len(self.tokenizer.encode(text)) // 2, 100)  # Example logic
        
        with torch.no_grad():
            inputs = self.tokenizer(text, return_tensors="pt")
            outputs = self.model.generate(
                inputs.input_ids.to(self.device),
                attention_mask=inputs.attention_mask.to(self.device),
                max_new_tokens=max_length,
                no_repeat_ngram_size=5,
                early_stopping=True,
                num_beams=2,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=2.0,
                temperature=temperature,  # Adjusted mood
                top_p=top_p,  # Adjusted mood
                do_sample=True,
                length_penalty=0.3
            )

            # Decode response and return
            response_outputs = outputs[:, len(inputs['input_ids'][0]):]
            response = self.tokenizer.batch_decode(response_outputs, skip_special_tokens=True)[0]
            return response

    def create_chat(self) -> Chat:
        return Chat(self)




In [13]:
# Load the ChatBot model
chatBot = ChatBot(model_path)

# Open a new Chat
conversation = chatBot.create_chat()
# Example usage

# Set topic
conversation.set_topic("Health and Fitness")

# Adjust chatbot mood
conversation.set_mood(temperature=0.7, top_p=0.8)

# Converse with bot

message = 'Should I excercise daily?'
print(f'User: {message}')

response = conversation.send(message)
print(f'Chatbot: {response}')

message = 'Thank you'
print(f'User: {message}')

response = conversation.send(message)
print(f'Chatbot: {response}')

User: Should I excercise daily?
Chatbot: Yes, you should. You're going to need a lot
User: Thank you
Chatbot: I'm not sure what that means but it's the same
